# Data Generation for Technical Report

In [33]:
# imports

from rethon import GlobalREEnsembleGenerator, ReflectiveEquilibrium, REState, StandardGlobalReflectiveEquilibrium
from rethon.util import standard_model_params_varied_alphas
from tau.util import create_random_argument_list
from tau.util import random_positions

from tau import StandardPosition, DAGDialecticalStructure, DialecticalStructure, BitarrayPosition, BDDDialecticalStructure
from tau.util import inferential_density

from os import getcwd, path
from typing import List

import pandas as pd
from pandas import Series

from ast import literal_eval
import tarfile

from math import floor

In [25]:
# relative path to data directory
output_directory = path.join(path.dirname(getcwd()), "data")

In [26]:
# model variants

implementations = [# quadratic default systematicity
                   {'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'rethon',
                    'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'},
    
                   # linear default systematicity
                   {'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'restudies.model_variations',
                    'reflective_equilibrium_class_name': 'GlobalReflectiveEquilibriumLinearDefaultSystematicity'},
    
                   # quadratic pure systematicity
                   {'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'restudies.model_variations',
                    'reflective_equilibrium_class_name': 'GlobalReflectiveEquilibriumQuadraticPureSystematicity'},
    
                   # linear pure systematicity
                   {'tau_module_name': 'tau',
                    'position_class_name':'StandardPosition',
                    'dialectical_structure_class_name': 'DAGDialecticalStructure',
                    'rethon_module_name': 'restudies.model_variations',
                    'reflective_equilibrium_class_name': 'GlobalReflectiveEquilibriumLinearPureSystematicity'}]

In [27]:
model_parameters_list = []

# weight resolution: 0.10
for model_parameters in standard_model_params_varied_alphas(9):
    
    # rounding
    for weight_name in model_parameters["weights"].keys():
    
        model_parameters["weights"][weight_name] = round(model_parameters["weights"][weight_name],2)
        
    model_parameters_list.append(model_parameters)

model_parameters_list


[{'weights': {'account': 0.1, 'faithfulness': 0.8, 'systematicity': 0.1},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'faithfulness': 0.7, 'systematicity': 0.2},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'faithfulness': 0.6, 'systematicity': 0.3},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'faithfulness': 0.5, 'systematicity': 0.4},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'faithfulness': 0.4, 'systematicity': 0.5},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0]},
 {'weights': {'account': 0.1, 'faithfulness': 0.3, 'systematicity': 0.6},
  'account_penalties': [0.0, 0.3, 1.0, 1.0],
  'faithfulness_penalties': [0.

In [28]:
# dialectical structures

sentence_pool_sizes = [6,7,8,9]
n_dialectical_structures = 1
max_n_premises = 2


# initial commitments
n_init_coms = 1


# RE process
track_branches = False

In [35]:
# main loop

for n_sentences in sentence_pool_sizes:
    
    # dialectical structures in inferential density range
    arguments_list = []
    while len(arguments_list)<n_dialectical_structures:

        arguments = create_random_argument_list(n_arguments_min=n_sentences-2, 
                                                n_arguments_max=n_sentences+1,
                                                n_sentences=n_sentences, 
                                                n_premises_max=max_n_premises)
        
        dia = BDDDialecticalStructure(n_sentences, arguments)
        
        # check inferential density and usage of all sentences
        if (0.15<=inferential_density(dia)<=0.5) and (len(set(abs(s) for arg in arguments for s in arg))==n_sentences):
            
            arguments_list.append(arguments)
            print(arguments)
            
    # initial commitments

    coms_list = random_positions(n_sentences, n_init_coms)


    ensemble_gen = GlobalREEnsembleGenerator(arguments_list = arguments_list, 
                                             n_sentence_pool = n_sentences,
                                             initial_commitments_list = coms_list,
                                             model_parameters_list = model_parameters_list,
                                             implementations = implementations,
                                             create_branches = track_branches)


    ensemble_gen.ensemble_items_to_csv(
                                  output_file_name = 'test.csv',
                                  output_dir_name =  output_directory,
                                  archive = True, # save the csv as archived tar.gz
                                  save_preliminary_results = True, # will create preliminary csv-data sets 
                                  preliminary_results_interval = 100)

[[1, -6], [-4, -3, 6], [2, -6], [3, -6, 2], [-1, 5, -2], [-3, 2], [5, -4]]
2023-08-08 15:49:52,848 [INFO] rethon: Starting ensemble generation with 144 models runs (without branches)
[[-1, -3, 7], [5, 7], [-4, -1], [-6, 2, -5], [4, -2]]
2023-08-08 15:50:10,665 [INFO] rethon: Starting ensemble generation with 144 models runs (without branches)
[[-3, 6, 5], [-2, -6], [4, -3], [1, -5], [-8, 5], [-2, -6, -8], [7, 1, -4]]
2023-08-08 15:51:31,947 [INFO] rethon: Starting ensemble generation with 144 models runs (without branches)
[[1, 6], [7, -9, -6], [5, 6], [3, -2, 5], [-8, 9], [4, 2], [4, 5, 9], [1, 9], [8, 6, 4]]
2023-08-08 15:53:09,057 [INFO] rethon: Starting ensemble generation with 144 models runs (without branches)
